## NA imputation 

In [1]:
import jusipy
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import missingno as msno
import copy
from fast_imputation import fast_imputation

#### Loading...

In [2]:
all_points = pd.read_pickle('data/all_points.pkl')

In [3]:
all_points.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22107 entries, 0 to 19999
Data columns (total 6 columns):
lat         22107 non-null object
long        22107 non-null float64
country     22107 non-null object
year        22107 non-null object
positive    22107 non-null int64
random      22107 non-null int64
dtypes: float64(1), int64(2), object(3)
memory usage: 1.2+ MB


In [4]:
A_country = jusipy.country_features.All()

Loading WorldBank(/home/marios/Desktop/Personal/repos/JuSiPy_Asser/jusipy/country_features/data/SD_API_15_DS2_en_csv_v2_10186395.csv)                                                                                                    

In [5]:
country_features = jusipy.country_features.get(all_points, A_country, fuzzy=50)

In [6]:
reg_prop = pd.read_excel('Registering Property.xlsx')

In [14]:
reg_prop[['Economy','Land dispute resolution index (0–8)']]

,Economy,Land dispute resolution index (0–8)
0,Afghanistan,3.0
1,Albania,7.0
2,Algeria,5.0
3,Angola,4.0
4,Antigua and Barbuda,7.5
5,Argentina,4.0
6,Armenia,7.0
7,Australia,5.5
8,Austria,5.0
9,Azerbaijan,7.5


-------------------------------------------------------------------------------

### NA percentage (in red)

In [20]:
# somehow unconsciouly it came out like the French flag..

In [21]:
fields = ['WB_ARD__AG_SRF_TOTL_K2', 'WB_SE__WB_NY_GDP_MKTP_KD_ZG', 'WB_ARD__SP_RUR_TOTL', 'WB_ARD__EG_ELC_ACCS_RU_ZS', 'WB_ARD__AG_LND_TOTL_RU_K2',
          'WB_ARD__AG_LND_FRST_K2', 'WB_ARD__AG_LND_ARBL_ZS', 'WB_ARD__AG_LND_AGRI_K2', 'WB_ARD__AG_CON_FERT_ZS','WB_ARD__AG_LND_PRCP_MM','WB_ARD__EN_ATM_METH_AG_KT_CE',
         'WB_ARD__EN_ATM_NOXE_AG_KT_CE','WB_ARD__NV_AGR_TOTL_ZS','WB_ARD__TM_VAL_AGRI_ZS_UN','WB_ARD__TX_VAL_AGRI_ZS_UN','WB_ARD__AG_LND_TOTL_K2',
         'WB_ARD__SL_AGR_EMPL_ZS','WB_SPL__SL_TLF_0714_ZS', 'TI_CPI__TI_CPI_CPISN']
fields_3 = list(country_features.keys())
#for x in fields_3:
#    df = country_features[x]
#    if x in fields:
#        print('\033[94m'+x+': '+'\033[92m'+A_country.description[x]+' has '+str(sum(df.isna()))+' NA '+'\033[91m'+str(sum(df.isna())/len(df)))
#    else:
#        print('\033[94m'+x+': '+'\033[0m'+A_country.description[x]+' has '+str(sum(df.isna()))+' NA '+'\033[91m'+str(sum(df.isna())/len(df)))
nas = []
nas_per = []
des = []
for x in fields_3:
    df = country_features[x]
    nas.append(sum(df.isna()))
    nas_per.append(sum(df.isna())/len(df))
    des.append(A_country.description[x])

    ###Printing the ones with NA < 10%
# DF = pd.DataFrame({'Indicator': fields_3, 'Description': des, 'Percentage of NA':nas_per})
# DF[DF["Percentage of NA"]<0.1]

# def color_field(val):
 
#     color = 'green' if val in fields else None
#     return 'color: %s' % color

# s = DF.style.applymap(color_field)
# s

In [28]:
fields = ['WB_ARD__AG_SRF_TOTL_K2', 'WB_SE__WB_NY_GDP_MKTP_KD_ZG', 'WB_ARD__SP_RUR_TOTL', 'WB_ARD__EG_ELC_ACCS_RU_ZS', 'WB_ARD__AG_LND_TOTL_RU_K2',
          'WB_ARD__AG_LND_FRST_K2', 'WB_ARD__AG_LND_ARBL_ZS', 'WB_ARD__AG_LND_AGRI_K2', 'WB_ARD__AG_CON_FERT_ZS','WB_ARD__AG_LND_PRCP_MM','WB_ARD__EN_ATM_METH_AG_KT_CE',
         'WB_ARD__EN_ATM_NOXE_AG_KT_CE','WB_ARD__NV_AGR_TOTL_ZS','WB_ARD__TM_VAL_AGRI_ZS_UN','WB_ARD__TX_VAL_AGRI_ZS_UN','WB_ARD__AG_LND_TOTL_K2',
         'WB_ARD__SL_AGR_EMPL_ZS']
imputed_ = []
for x in fields:
    df_temp = country_features[x]
    imputed_.append(fast_imputation(df_temp))
